In [ ]:
from modules import *

In [ ]:
model = UNet(in_channels=1, out_channels=1, Transpose=True)

In [ ]:
model

In [ ]:
tensor_size = (1, 1, 572, 572)
test_tensor = torch.randn(tensor_size)
opt = model(test_tensor)

In [ ]:
test_tensor.size()

In [ ]:
opt.size()

In [ ]:
from modules import *

model = VanillaVAE(
    in_channels=3,
    latent_dim=128
)


In [ ]:
model

In [ ]:
import torch
import torch.nn as nn

# Batch_size x Channels x Height x Width
tensor_size = (3, 3, 28, 28)
tensor = torch.randn(tensor_size)
print(tensor.size())

In [ ]:
mu, log_var = model.encode(tensor)
print(mu.size())
print(log_var.size())

In [ ]:
result = model.encode(tensor)
print(result.size())

In [ ]:
test = torch.flatten(result, start_dim=1)
print(test.size())